In [1]:
import pandas as pd
import json
from math import radians
import numpy as np

In [2]:

ruta='pisos_limpios.csv'
dfknn = pd.read_csv(ruta,sep=',')
dfknn.columns
dfknn.set_index('uv', inplace=True)
with open('pesos.json') as archivo:
            opciones = json.load(archivo)
filtros=opciones['filtros']
filtros = filtros[2:4]
binarias =opciones['binarias']
binarias

{'portero': 1,
 'ascensor': 2,
 'parque_infantil': 3,
 'terraza': 1,
 'trastero': 3,
 'cuarto_de_basura': 1,
 'zonas_comunes': 1,
 'piscina': 3,
 'garaje': 5,
 'calefaccion': 1,
 'jardin': 5,
 'aire_acondicionado': 1,
 'ns': 3,
 'eo': 1}

In [3]:
df = dfknn.copy()
target = df.iloc[15]
target=pd.DataFrame(target).transpose()

In [4]:
def haversine(d, t):
    t = t.loc[t.index.repeat(len(d))]
    indice = d.index
    t.reset_index(drop=True, inplace=True)
    d.reset_index(drop=True, inplace=True)
    
    lon1 = d.long
    lat1 = d.lat
    lon2 = t.long
    lat2 = t.lat
    

    lat1 = lat1.map(radians)
    lon1 = lon1.map(radians)
    lat2 = lat2.map(radians)
    lon2 = lon2.map(radians)


    #lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    km = 6371* c
    return km

In [5]:
def calc_ascensor(da,dp,ta,tp):
    try:
        if da ==1 and ta ==1: 
            distancia_ascensor = 10
        elif da ==0 and ta == 0: 
            distancia_ascensor= (dp-tp) * ((dp-tp)>0)
        elif da==0 and ta==1:
            if dp < 2: 
                distancia_ascensor = 9
            else:
                distancia_ascensor = dp
        elif da==1 and ta==0:
            distancia_ascensor = 8
        else:
            distancia_ascensor= 7
    except:
        distancia_ascensor=6
    return distancia_ascensor

In [6]:
target.iloc[0]['intext']

3

In [7]:
filtros =['provincia']

In [8]:

for f in filtros:
    #if f=='bajo':
    #    indice = df[ df[planta] == 0 ].index
    #else:
    dato = target.iloc[0][f]

    indice = df[ df[f] != dato ].index
    df.drop(indice, axis=0, inplace=True)
    df.drop(f,axis=1, inplace=True)

###calculamos geo
distancia_geo = haversine(df[['long','lat']], target[['long','lat']])
###calculamos geo
distancia_ascensor=df.apply(lambda row: calc_ascensor(row['ascensor'], row['planta'], target.ascensor[0],target.planta[0]),axis=1)

df_binarias=df[binarias]
target_binarias = target[binarias].loc[target.index.repeat(len(df_binarias))]
target_binarias.index=df_binarias.index
iguales=target_binarias==df_binarias
distintas=target_binarias!=df_binarias
s_binarias=pd.Series(binarias)
similitud_binarias= iguales.dot(s_binarias) + distintas.dot(1-s_binarias)

In [9]:
similitud_binarias

uv
UV/2020/000003    30
UV/2020/000004    21
UV/2020/000006    21
UV/2020/000007    26
UV/2020/000008    21
                  ..
UV/2021/000443    20
UV/2021/000444    22
UV/2021/000448    23
UV/2021/000456    27
UV/2021/000461    22
Length: 144, dtype: int64

In [10]:
similitud_binarias= iguales.dot(binarias.values)

IndexError: tuple index out of range